In [2]:
# Install dependencies
!pip install torchaudio qdrant-client --quiet

# Download metadata and audio
!wget -nc https://os.unil.cloud.switch.ch/fma/fma_small.zip -O fma_small.zip
#!wget -nc https://os.unil.cloud.switch.ch/fma/fma_metadata.zip -O fma_metadata.zip

# Unzip (audio: fma_small, metadata: CSVs)
!unzip -q -n fma_small.zip -d ./fma_small
#!unzip -q -n fma_metadata.zip -d ./fma_metadata

File ‘fma_small.zip’ already there; not retrieving.


In [ ]:
import pandas as pd

tracks = pd.read_csv("fma_metadata/fma_metadata/tracks.csv", index_col=0, header=[0,1])
genres = pd.read_csv("fma_metadata/fma_metadata/genres.csv", index_col=0)

# Filter only small set
subset = tracks['set', 'subset'] == 'small'
small_tracks = tracks[subset]

print("Total FMA-Small tracks:", len(small_tracks))
small_tracks.head()

Total FMA-Small tracks: 8000


album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00      NaN   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00      NaN   
140             1  2008-11-26 01:49:59  2007-05-22 00:00:00      NaN   
141             0  2008-11-26 01:49:57  2009-01-16 00:00:00      NaN   

                                                                           \
         favorites  id                                        information   
track_id                                                                    
2                4   1                                            <p></p>   
5                4   1                                            <p></p>   
10               4   6                                                NaN   
140              1  61  <p>Alec K. Redfearn &amp; The Eyesores: Ellen ...   
141              1  60  <p>A full ensamble of strings, drums, electron...   

                                                       ...       track  \
         listens                        producer tags  ... information   
track_id                                               ...               
2           6073                             NaN   []  ...         NaN   
5           6073                             NaN   []  ...         NaN   
10         47632                             NaN   []  ...         NaN   
140         1300  Alec K. Refearn, Rob Pemberton   []  ...         NaN   
141         1304                             NaN   []  ...         NaN   

                                 \
         interest language_code   
track_id                          
2            4656            en   
5            1933            en   
10          54881            en   
140          1593            en   
141           839            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293      NaN   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151      NaN   
10        Attribution-NonCommercial-NoDerivatives (aka M...   50135      NaN   
140       Attribution-Noncommercial-No Derivative Works ...    1299      NaN   
141       Attribution-Noncommercial-No Derivative Works ...     725      NaN   

                                                    
         number publisher tags               title  
track_id                                            
2             3       NaN   []                Food  
5             6       NaN   []          This World  
10            1       NaN   []             Freeway  
140           2       NaN   []  Queen Of The Wires  
141           4       NaN   []                Ohio  

[5 rows x 52 columns]

In [3]:
SAMPLE_RATE = 16000
N_MELS = 64
CLIP_DURATION = 10     # seconds (small for training batches, large enough for music context)
N_SAMPLES = SAMPLE_RATE * CLIP_DURATION

In [ ]:
import os
import torch
import torchaudio
from tqdm import tqdm
import glob

#DATASET_PATH = "fma_small/fma_small"
#CACHE_PATH = "/content/fma_cache"
DATASET_PATH = "/kaggle/working/fma_small/fma_small"
CACHE_PATH   = "/kaggle/working/fma_cache"
os.makedirs(CACHE_PATH, exist_ok=True)

# Parameters
clip_duration = CLIP_DURATION       # seconds
sample_rate = SAMPLE_RATE
n_mels = N_MELS
n_samples = clip_duration * sample_rate

# Mel transform
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=1024,
    hop_length=512,
    n_mels=n_mels
)

# Gather all song paths
song_files = [p for p in glob.glob(os.path.join(DATASET_PATH, "*/*.mp3"))]
song_dirs = {f"song_{i:06d}": p for i, p in enumerate(song_files)}

# Precompute
for song_id, path in tqdm(song_dirs.items(), desc="Caching spectrograms"):
    cache_file = os.path.join(CACHE_PATH, f"{song_id}.pt")
    if os.path.exists(cache_file):
        continue  # skip if already cached

    try:
        waveform, sr = torchaudio.load(path)
    except Exception as e:
        print(f"Skipping problematic file {song_id}: {e}")
        continue  # skip problematic file
    if waveform.size(1) < 1000:
      print(f"Skipping too short file: {song_id}")
      continue
    if sr != sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, sample_rate)
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)  # mono

    # Take random clip_duration seconds (or pad if short)
    if waveform.size(1) < n_samples:
        pad = n_samples - waveform.size(1)
        waveform = torch.nn.functional.pad(waveform, (0, pad))
    else:
        max_start = waveform.size(1) - N_SAMPLES
        start = torch.randint(0, max_start + 1, (1,)).item()
        waveform = waveform[:, start:start + N_SAMPLES]

    mel = mel_transform(waveform)
    log_mel = torch.log1p(mel)
    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-6)
    log_mel = log_mel.contiguous()

    torch.save(log_mel, cache_file)


In [5]:
import torch
from torch.utils.data import Dataset
import random
import os

class SongTripletDatasetCached(Dataset):
    def __init__(self, cache_dir, clip_duration=5, sample_rate=16000, hop_length=512):
        """
        clip_duration: seconds of each anchor/positive/negative
        """
        self.cache_dir = cache_dir
        self.cache_files = [os.path.join(cache_dir, f) for f in os.listdir(cache_dir)]
        self.song_ids = [os.path.basename(f).split(".pt")[0] for f in self.cache_files]

        # Compute number of mel frames per clip
        self.frames_per_sec = sample_rate / hop_length
        self.n_frames = int(clip_duration * self.frames_per_sec)

    def __len__(self):
        # You can use the number of cached songs, or any large number for random triplet sampling
        return 5 * len(self.song_ids)

    def random_crop(self, mel):
        total_frames = mel.size(1)
        if total_frames <= self.n_frames:
            return mel  # too short, return as is
        start = random.randint(0, total_frames - self.n_frames)
        return mel[:, start:start + self.n_frames]

    def __getitem__(self, idx):
        # --- Anchor ---
        anchor_id = random.choice(self.song_ids)
        anchor_mel = torch.load(os.path.join(self.cache_dir, f"{anchor_id}.pt"))
        anchor = self.random_crop(anchor_mel)

        # --- Positive (same song, different clip) ---
        positive_mel = torch.load(os.path.join(self.cache_dir, f"{anchor_id}.pt"))
        positive = self.random_crop(positive_mel)

        # --- Negative (different song) ---
        neg_id = random.choice([s for s in self.song_ids if s != anchor_id])
        negative_mel = torch.load(os.path.join(self.cache_dir, f"{neg_id}.pt"))
        negative = self.random_crop(negative_mel)

        return anchor, positive, negative


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import Dataset, DataLoader
import random
import os

# ----------------------------
# Embedding Network
# ----------------------------
"""
class AudioEmbeddingNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(128, embedding_dim)

    def forward(self, x):
        # x: (B, 1, n_mels, time)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.normalize(x, p=2, dim=1)  # L2-normalized embeddings
"""

class AudioEmbeddingNet(nn.Module):
    def __init__(self, n_mels=64, embedding_dim=128, lstm_hidden=128, lstm_layers=1):
        super(AudioEmbeddingNet, self).__init__()

        # ------------------------------
        # 1️⃣ CNN feature extractor
        # ------------------------------
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(3,3), padding=(1,1)),  # input: (B,1,n_mels,T)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),  # downsample (n_mels//2, T//2)

            nn.Conv2d(32, 64, kernel_size=(3,3), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),  # downsample further

            nn.Conv2d(64, 128, kernel_size=(3,3), padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        # ------------------------------
        # 2️⃣ BiLSTM for temporal modeling
        # ------------------------------
        # After CNN, we collapse the mel-dim, keep time as sequence
        self.lstm = nn.LSTM(
            input_size=128*(n_mels//4),  # depends on pooling
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True
        )

        # ------------------------------
        # 3️⃣ Fully connected embedding
        # ------------------------------
        self.fc = nn.Linear(2*lstm_hidden, embedding_dim)  # BiLSTM -> embedding

    def forward(self, x):
        """
        x: (B, 1, n_mels, T) log-mel spectrogram
        """
        B, C, H, W = x.size()
        x = self.cnn(x)  # (B, 128, H', W')
        x = x.permute(0, 3, 1, 2)  # (B, W', C, H')
        x = x.contiguous().view(B, x.size(1), -1)  # flatten mel+channels -> seq_len x features

        lstm_out, _ = self.lstm(x)  # (B, seq_len, 2*lstm_hidden)
        # Take mean over time dimension
        embedding = lstm_out.mean(dim=1)
        embedding = self.fc(embedding)  # final embedding
        embedding = F.normalize(embedding, p=2, dim=1)  # optional: normalize

        return embedding

# Triplet wrapper
class TripletNetwork(nn.Module):
    def __init__(self, embedding_net):
        super().__init__()
        self.embedding_net = embedding_net

    def forward(self, anchor, positive, negative):
        return (self.embedding_net(anchor),
                self.embedding_net(positive),
                self.embedding_net(negative))


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
import os

# 🔹 1. Train/Val split
def create_loaders(dataset, batch_size=32, val_split=0.2):
    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    return train_loader, val_loader


# 🔹 2. Training loop with early stopping + scheduler
def train_model(model, train_loader, val_loader, n_epochs=50, patience=5, save_path="best_model.pt"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    criterion = nn.TripletMarginLoss(margin=1.0, p=2)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2)

    best_val_loss = np.inf
    patience_counter = 0

    for epoch in range(1, n_epochs + 1):
        # --- Training ---
        model.train()
        total_train_loss = 0
        for anchor, positive, negative in train_loader:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            optimizer.zero_grad()

            anchor_out, positive_out, negative_out = model(anchor, positive, negative)

            loss = criterion(anchor_out, positive_out, negative_out)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # --- Validation ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for anchor, positive, negative in val_loader:
                anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
                anchor_out, positive_out, negative_out = model(anchor, positive, negative)
                val_loss = criterion(anchor_out, positive_out, negative_out)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # 🔹 Scheduler step
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch:03d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

        # --- Check early stopping ---
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), save_path)  # save best model
            print(f"  ✅ New best model saved (val_loss={best_val_loss:.4f})")
        else:
            patience_counter += 1
            print(f"  ⚠️ No improvement (patience {patience_counter}/{patience})")

        if patience_counter >= patience:
            print("⏹️ Early stopping triggered")
            break

    # Load best model before returning
    model.load_state_dict(torch.load(save_path))
    print("🔄 Best model reloaded from checkpoint")
    return model


In [9]:
import os
import glob
from torch.utils.data import DataLoader, random_split

dataset = SongTripletDatasetCached(CACHE_PATH)
val_split = 0.1
val_size = int(len(dataset) * val_split)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

# Model
model = TripletNetwork(AudioEmbeddingNet(embedding_dim=128))

# If more than 1 GPU is available, wrap with DataParallel
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = torch.nn.DataParallel(model)
model = model.cuda()  # move to GPU(s)

# Train
best_model = train_model(model, train_loader, val_loader, n_epochs=20, patience=5, save_path="triplet_best.pt")


Epoch 001 | Train Loss: 0.0086 | Val Loss: 0.0043 | LR: 0.001000
  ✅ New best model saved (val_loss=0.0043)
Epoch 002 | Train Loss: 0.0026 | Val Loss: 0.0061 | LR: 0.001000
  ⚠️ No improvement (patience 1/5)
Epoch 003 | Train Loss: 0.0021 | Val Loss: 0.0023 | LR: 0.001000
  ✅ New best model saved (val_loss=0.0023)
Epoch 004 | Train Loss: 0.0015 | Val Loss: 0.0012 | LR: 0.001000
  ✅ New best model saved (val_loss=0.0012)
Epoch 005 | Train Loss: 0.0017 | Val Loss: 0.0024 | LR: 0.001000
  ⚠️ No improvement (patience 1/5)
Epoch 006 | Train Loss: 0.0014 | Val Loss: 0.0014 | LR: 0.001000
  ⚠️ No improvement (patience 2/5)
Epoch 007 | Train Loss: 0.0011 | Val Loss: 0.0010 | LR: 0.001000
  ✅ New best model saved (val_loss=0.0010)
Epoch 008 | Train Loss: 0.0009 | Val Loss: 0.0011 | LR: 0.001000
  ⚠️ No improvement (patience 1/5)
Epoch 009 | Train Loss: 0.0010 | Val Loss: 0.0006 | LR: 0.001000
  ✅ New best model saved (val_loss=0.0006)
Epoch 010 | Train Loss: 0.0007 | Val Loss: 0.0008 | LR: 0.00

In [16]:
import torch
import numpy as np
import random
import os

# Constants
SAMPLE_RATE = 16000
HOP_LENGTH = 512
N_MELS = 64
CROP_DURATION = 5  # seconds for random crop
CROP_FRAMES = int(CROP_DURATION * SAMPLE_RATE / HOP_LENGTH)  # frames in mel spectrogram

# Mel spectrogram transform
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=N_MELS
)

def preprocess_mp3_file(path, device="cpu"):
    """
    path: path to an .mp3 file
    Returns a random 5-second crop of log-mel spectrogram, normalized
    """
    try:
        waveform, sr = torchaudio.load(path)
    except Exception as e:
        print(f"Skipping {song_id}: {e}")
        return None  
    if waveform.size(1) < 1000:
      print(f"Skipping too short file: {song_id}")
      return None
    if sr != sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, sample_rate)
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)  # mono

    mel = mel_transform(waveform)

    # Random crop
    total_frames = mel.size(1)
    if total_frames <= CROP_FRAMES:
        cropped = mel
    else:
        start = random.randint(0, total_frames - CROP_FRAMES)
        cropped = mel[:, start:start + CROP_FRAMES]
        
    log_mel = torch.log1p(cropped)
    log_mel = (log_mel - log_mel.mean()) / (log_mel.std() + 1e-6)
    
    return log_mel.unsqueeze(0).to(device).contiguous()  # (1, n_mels, time)
    
def preprocess_cached_file(path, device="cpu"):
    """
    path: path to cached .pt file (10-second log-mel spectrogram)
    Returns a random 5-second crop, normalized
    """
    # Load cached mel spectrogram
    mel = torch.load(path)  # (n_mels, time_frames)

    # Random crop
    total_frames = mel.size(1)
    if total_frames <= CROP_FRAMES:
        cropped = mel
    else:
        start = random.randint(0, total_frames - CROP_FRAMES)
        cropped = mel[:, start:start + CROP_FRAMES]

    # Normalize
    cropped = (cropped - cropped.mean()) / (cropped.std() + 1e-6)

    return cropped.unsqueeze(0).to(device).contiguous()  # (1, n_mels, time)

def embed_song(embedding_net, path, device="cpu"):
    """
    embedding_net: your neural net
    path: path to cached mel .pt file
    """
    embedding_net.eval()
    with torch.no_grad():
        #spec = preprocess_cached_file(path, device)
        spec = preprocess_mp3_file(path, device)
        emb = embedding_net(spec)  # (1, embedding_dim)
        return emb.squeeze(0).cpu().numpy().astype(np.float32)


In [ ]:
from qdrant_client import QdrantClient

# Connect to Qdrant
client = QdrantClient(":memory:")

# Delete collection
client.delete_collection(collection_name="songs")

print("✅ Collection 'songs' deleted")


✅ Collection 'songs' deleted


In [17]:
import os
import glob
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

# Qdrant client (local or cloud)
#client = QdrantClient(host="localhost", port=6333)
client = QdrantClient(":memory:")  # only works in current runtime, no network needed

# Pick first 1000 MP3s MELs from FMA-Small
#DATASET_PATH = "fma_small/fma_small"
#CACHE_PATH = "/content/fma_cache"
DATASET_PATH = "/kaggle/working/fma_small/fma_small"
CACHE_PATH   = "/kaggle/working/fma_cache"
#song_files = sorted(glob.glob(os.path.join(CACHE_PATH, "*/*.pt")))[:1000]
song_files = sorted(glob.glob(os.path.join(DATASET_PATH, "*/*.mp3")))[:1000]

# Create or reset collection
client.recreate_collection(
    collection_name="songs",
    vectors_config=rest.VectorParams(size=128, distance="Cosine")
)

embedding_net = AudioEmbeddingNet(embedding_dim=128)
# Load weights from TripletNetwork
state_dict = torch.load("triplet_best.pt", map_location="cpu")
# If keys are prefixed with 'embedding_net.', strip them
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith("embedding_net."):
        new_state_dict[k.replace("embedding_net.", "")] = v

embedding_net.load_state_dict(new_state_dict)

# Upsert songs
points = []
for idx, path in enumerate(tqdm(song_files, desc="Indexing songs")):
    try:
        emb = embed_song(embedding_net, path, device="cpu")  # use embedding_net here
        emb = emb / np.linalg.norm(emb)
        points.append(
            rest.PointStruct(
                id=idx,
                vector=emb.tolist(),
                payload={"track": os.path.basename(path)}
            )
        )
    except Exception as e:
        print(f"❌ Failed {path}: {e}")

# Bulk insert
if points:
    client.upsert(
        collection_name="songs",
        points=points
    )

print(f"✅ Inserted {len(points)} songs into Qdrant")


/tmp/ipykernel_36/2078873022.py:20: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
Indexing songs: 100%|██████████| 1000/1000 [01:47<00:00,  9.34it/s]


✅ Inserted 1000 songs into Qdrant


In [19]:
query_path = song_files[512]  # pick any > 1000
query_emb = embed_song(embedding_net, query_path)
query_emb = query_emb / np.linalg.norm(query_emb)

results = client.search(
    collection_name="songs",
    query_vector=query_emb.tolist(),
    limit=3
)

print("\n🔎 Query:", os.path.basename(query_path))
for hit in results:
    print(f"Match: {hit.payload['track']} (score={hit.score:.3f})")



🔎 Query: 011764.mp3
Match: 011764.mp3 (score=1.000)
Match: 004232.mp3 (score=0.525)
Match: 007376.mp3 (score=0.480)


/tmp/ipykernel_36/179024995.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [30]:
SONG_NAME = "freedom.mp3"
SONG_PATH = "/kaggle/input/test-songs/freedom.mp3"

In [ ]:
from google.colab import drive
import shutil
import os

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Set the path to your file in Drive
#drive_file_path = "/content/drive/MyDrive/freedom.mp3"
#local_path = "/content/freedom.mp3"
drive_file_path = f"/content/drive/MyDrive/{SONG_NAME}"
#local_path = f"/content/{SONG_NAME}"
local_path = f"/kaggle/working/{SONG_NAME}"


# 3️⃣ Copy to Colab working directory
shutil.copy(drive_file_path, local_path)

# 4️⃣ Check
assert os.path.exists(local_path), "File not found!"
#print(f"✅ freedom.mp3 copied to {local_path}")
print(f"✅ {SONG_NAME} copied to {local_path}")


In [31]:
import torchaudio
import torch
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct

#song_path = "/content/freedom.mp3"  # path to Beyoncé's Freedom
#song_path = "/content/halo.mp3"  # path to Beyoncé's Halo
collection_name = "songs"

#embedding_freedom = embed_song(embedding_net, song_path, device="cpu")
embedding_freedom = embed_song(embedding_net, SONG_PATH, device="cpu")
embedding_freedom = embedding_freedom / np.linalg.norm(embedding_freedom)  # normalize

# ------------------------------
# 3️⃣ Connect to Qdrant
# ------------------------------
qdrant_client = client

# Only create the collection if it doesn't exist
if not qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config={"size": 128, "distance": "Cosine"}
    )

# ------------------------------
# 4️⃣ Upsert Beyoncé's Freedom or Halo
# ------------------------------
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(
            id=1000000,  # unique ID
            vector=embedding_freedom.tolist(),
            payload={"track": "Beyonce Freedom"}
            #payload={"track": "Beyonce Halo"}
        )
    ]
)

#print("✅ Inserted 'Beyonce Freedom' into Qdrant")
print(f"✅ Inserted '{SONG_NAME}' into Qdrant")


✅ Inserted 'freedom.mp3' into Qdrant


In [32]:
#query_path = "/content/freedom.mp3"
#query_path = "/content/halo.mp3"
query_path = SONG_PATH
query_emb = embed_song(embedding_net, query_path)
query_emb = query_emb / np.linalg.norm(query_emb)

results = client.search(
    collection_name="songs",
    query_vector=query_emb.tolist(),
    limit=3
)

print("\n🔎 Query:", os.path.basename(query_path))
for hit in results:
    print(f"Match: {hit.payload['track']} (score={hit.score:.3f})")


🔎 Query: freedom.mp3
Match: Beyonce Freedom (score=1.000)
Match: Beyonce Halo (score=0.789)
Match: 001642.mp3 (score=0.607)


/tmp/ipykernel_36/3672160408.py:7: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
